In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os

from overlapped_classes import overlapped_classes
import json
from PIL import Image
import scipy

In [2]:
with open("../mappings/objectnet_to_imagenet_1k.json") as f:
    data = json.load(f)

with open("../mappings/pytorch_to_imagenet_2012_id.json") as f:
    idxMap = json.load(f)
        
with open("../mappings/folder_to_objectnet_label.json") as f:
    folderMap = json.load(f)

with open('imagenet_classes.txt') as f:
# with open('../mappings/imagenet_to_label_2012_v2.txt') as f:    
    labels = [line.strip() for line in f.readlines()]    

In [3]:
# inverting the folderMap
dirMap = {}
for u, v in folderMap.items():
    dirMap[v]= u

In [36]:
# dirMap

In [ ]:
# help(json.load)
# for i in overlapped_classes:
#     if not data[i] in labels:
#         print(f"{i} : {data[i]}, {data[i] in labels}")

In [ ]:
# overlapped_classes
# data['Alarm clock']
# idxMap

In [ ]:
# for i in overlapped_classes:
#     print(data[i] in labels)
#     print(f"{i} : {data[i]}")
del model

In [62]:
from torchvision import models
dir(models)



model = models.alexnet(pretrained=True)
# model = models.resnet152(pretrained=True)
# model = models.inception_v3(pretrained=True)
# model = models.googlenet(pretrained=True)

model.eval()


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [59]:
def predict_image(image_path, box, draw=False):
    # Define transformations for the image, should (note that imagenet models are trained with image size 224)
    transform = transforms.Compose([
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))    

    ])
    
    
#     transform = transforms.Compose([            #[1]
#     #  transforms.Resize(256),                    #[2]
#     #  transforms.CenterCrop(224),                #[3]
#      transforms.ToTensor(),                     #[4]
#     #  transforms.Normalize(                      #[5]
#     #  mean=[0.485, 0.456, 0.406],                #[6]
#     #  std=[0.229, 0.224, 0.225]                  #[7]
#     #  )
#             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))    
#     ])
    
#     print("Prediction in progress")
    img = Image.open(image_path).convert("RGB")

    
    xl, yl, xr, yr = box
    img = img.crop((xl, yl, xr, yr))   #((left, top, right, bottom)) 
    img = img.resize((224, 224))

    img_t = transform(img).float()

#     img_t = img_t.permute((0,1,2))
#     img_t = torch.transpose(img_t, 0, 1)  # rotate
    
    if draw:
        plt.imshow(img) #img_t.permute((2,1,0)) )
    # ##    plt.imshow(torch.transpose(img_t.permute((1,2,0)),  0, 1))
        plt.show()

#     print(img_t.shape)
    # Preprocess the image
#     image_tensor = transformation(image).float()

    # Add an extra batch dimension since pytorch treats all images as batches
#     image_tensor = image_tensor.unsqueeze_(0)
    image_tensor = img_t.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()

    # Turn the input into a Variable
#     input = Variable(image_tensor)

    # Predict the class of the image
    output = model(image_tensor)

#     index = output.data.numpy().argmax()  # top 1
    _, indices = torch.sort(output.data, descending=True)
#     [(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]
    return indices[0][:5] # index

In [41]:
len(data)
dirMap[category]


'alarm_clock'

In [63]:
# category = 'Banana'

res_top1 = []
res_top5 = []

for n, category in enumerate(data):
#     if n > 1:
#         break
    txtfile = '../' + dirMap[category] + '.txt'
    if not os.path.exists(txtfile):
        continue
    
    with open(txtfile) as f:
        boxes = [line.strip() for line in f.readlines()]    

#     preds = []
    count = [0, 0]
    lines = 0
    for im in boxes[0:]:
        ss = im.split(' ')
        if len(ss) > 1:
            lines += 1
            coords = (int(i) for i in ss[1:] if i)
            idx = predict_image(os.path.join('../images/' + dirMap[category] + '/', ss[0]), coords, False)
            
            # top 1
            count[0] += 1 if labels[idx[0]] in data[category] else 0  
            
            # top 5
            flag = False
            for i in idx[0:]:
    #             print(labels[i], end = ' ')
#                 predLabels = labels[idxMap[str(i.tolist())]]
#                 print(i, labels[i])
                  flag = flag or (labels[i] in data[category])
            count[1] += 1 if flag else 0
#             print('\n')
#             preds.append(idx)



    accs = np.array(count)*100/lines
    
    print(f"{n} -> {category}: top 1: {accs[0]}  -  top 5: {accs[1]}      [num imgs: {lines}]")  

    res_top1.append(accs[0])        
    res_top5.append(accs[1])


0 -> Alarm clock: top 1: 6.25  -  top 5: 20.13888888888889      [num imgs: 144]
1 -> Backpack: top 1: 27.659574468085108  -  top 5: 53.61702127659574      [num imgs: 235]
2 -> Banana: top 1: 44.255319148936174  -  top 5: 62.5531914893617      [num imgs: 235]
3 -> Band Aid: top 1: 20.081967213114755  -  top 5: 37.704918032786885      [num imgs: 244]
4 -> Basket: top 1: 5.524861878453039  -  top 5: 23.756906077348066      [num imgs: 181]
5 -> Bath towel: top 1: 10.06711409395973  -  top 5: 26.845637583892618      [num imgs: 149]
6 -> Beer bottle: top 1: 8.67579908675799  -  top 5: 17.80821917808219      [num imgs: 219]
7 -> Bench: top 1: 0.0  -  top 5: 3.0303030303030303      [num imgs: 132]
8 -> Bicycle: top 1: 20.408163265306122  -  top 5: 40.816326530612244      [num imgs: 147]
9 -> Binder (closed): top 1: 8.241758241758241  -  top 5: 25.274725274725274      [num imgs: 182]
10 -> Bottle cap: top 1: 8.870967741935484  -  top 5: 19.35483870967742      [num imgs: 248]
11 -> Bread loaf: t

In [64]:
print(sum(res_top1)/len(res_top1))
print(sum(res_top5)/len(res_top5))


13.996669092969494
30.32472008287954


In [ ]:
# model = alexnet
# for l in preds:
#     print(labels[l])
# preds 


In [ ]:
# 
if ss is ['']:
    print('ss')
len(ss)

In [ ]:
# Import Pillow

from PIL import Image
img = Image.open("../images/air_freshener/01d44a4b77b44a0.png").convert('RGB')

img = img.crop((xl, yl, xr, yr))   #((left, top, right, bottom)) 
img = img.resize((224, 224))


img_t = transform(img)#.float()

img_t = img_t.permute((2,1,0))

img_t = torch.transpose(img_t, 0, 1)

# np.clip
# img_t = torch.unsqueeze(img_t, 0)
# help(img_t.transpose)

plt.imshow(img_t)

# img = cv2.imread("../images/air_freshener/01d44a4b77b44a0.png", cv2.IMREAD_UNCHANGED)
# img = cv2cvLoadImage("../images/air_freshener/01d44a4b77b44a0.png", CV_LOAD_IMAGE_UNCHANGED)


In [ ]:
np.asarray(img_t).shape

In [ ]:
img_t[:10,:10].min()

In [ ]:
help(torch.transpose)

In [ ]:
# plt.show(img)
%matplotlib inline
img = np.array(img)
# img = img[:,:,::-1]






In [ ]:
with open('../air_freshener.txt') as f:
    boxes = [line.strip() for line in f.readlines()]    

In [ ]:
xl, yl, xr, yr = [int(i) for i in boxes[11].split(' ')[1:] if i]
img[yl:yr,xl:xr, :]

# std = np.array([0.229, 0.224, 0.225])
# inp = std * inp + mean
# inp = np.clip(inp, 0, 1)

img_t = transform().float()
batch_t = torch.unsqueeze(img, 0)


plt.imshow(batch_t)
# cv2.imshow('t', img[xl:xr, yl:yr,:])
# # help(cv2.imshow())
# cv2.waitKey(0)

# Image.crop()

In [ ]:
# cv2.destroyAllWindows()


In [ ]:
xl, yl, xr, yr

In [ ]:
batch_t.shape

In [ ]:
# resnet = models.resnet101(pretrained=True)

alexnet.eval()
img_a = img_t.permute((2,1,0))
batch_t = torch.unsqueeze(img_a, 0)
out = alexnet(batch_t)

print(out.shape)

In [ ]:
# out
batch_t.shape

In [ ]:
with open('imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

In [ ]:
_, index = torch.max(out, 1)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print(labels[index[0]], percentage[index[0]].item())

In [ ]:
_, indices = torch.sort(out, descending=True)
2
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

In [ ]:
index

In [4]:
for n, category in enumerate(data):
#     if n > 1:
#         break
    txtfile = '../' + dirMap[category] + '.txt'
    if not os.path.exists(txtfile):
        print(category)

Cellphone
Clothes hamper
Mixing / Salad Bowl
Monitor
Mug
Nail (fastener)
Necklace
Orange
Padlock
Paintbrush
Paper towel
Pen
Pill bottle
Pillow
Plastic bag
Plate
Soap dispenser
Sock
Soup Bowl
Spatula
Speaker
Still Camera
Strainer
Stuffed animal
Suit jacket
Sunglasses
Sweater
Swimming trunks
T-shirt
TV
Teapot
Wheel
Whistle
Wine bottle
Winter glove
Wok


In [9]:
newFolderMap = {}
for u, v in folderMap.items():
    newFolderMap[v] = u
    


In [21]:
for i in data: 
    if newFolderMap.get(i,0) == 0:
        print(i)

In [22]:
newFolderMap[i]
i

'Wok'

In [24]:
newFolderMap.get('Alarm clock',0)

'alarm_clock'

In [35]:
# len(set(newFolderMap.keys()) - set(data))
to_do = []
for j in set(newFolderMap.keys()) - set(data):
    txtfile = '../' + dirMap[j] + '.txt'
    if not os.path.exists(txtfile):
        to_do.append(j)
        
print(to_do)

['Step stool', 'Pill organizer', 'Mouthwash', 'Honey container', 'Key chain', 'Tanktop', 'Document folder (closed)', 'Tarp', 'Webcam', 'Floss container', 'Figurine or statue', 'Tongs', 'Suitcase', 'Skateboard', 'Teabag', 'Earring', 'Wrench', 'Squeeze bottle', 'Shoelace', 'Flashlight', 'Table knife', 'Hair brush', 'Eraser (white board)', 'Tote bag', 'Trash bag', 'Peeler', 'Extension cable', 'Tweezers', 'Dress pants', 'Nut for a screw', 'Paper plates', 'Notebook', 'Ice', 'Phone (landline)', 'Hairtie', 'Laptop charger', 'Makeup', 'Multitool', 'DVD player', 'Sponge', 'Walking cane', 'Ring', 'Kettle', 'Makeup brush', 'Placemat', 'Notepad', 'Deodorant', 'Playing cards', 'Spray bottle', 'Loofah', 'Napkin', 'Nightstand', 'Standing lamp', 'Plastic cup', 'Walker', 'Throw pillow', 'Tape measure', 'Nail clippers', 'Paint can', 'Oven mitts', 'Rock', 'Usb flash drive', 'Poster', 'Toothpaste', 'Video Camera', 'Rolling pin', 'Slipper', 'Flour container', 'Pepper shaker', 'Soap bar', 'Drinking straw', 

In [36]:
len(data)
len(to_do)

141

In [40]:
np.exp(-.12)

0.8869204367171575

In [39]:
np.exp(-.2)

0.8187307530779818